In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install opendatasets
!pip install pandas
!pip install seaborn --upgrade
!pip install sklearn --upgrade

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
import opendatasets as od
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [3]:
#od.download("https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques")

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
house_train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
house_test_df = pd.read_csv('/content/drive/MyDrive/test.csv')
house_Y_col = house_train_df.columns[-1]
house_X_col = house_train_df.columns[1:-1]
#titanic_X_col = titanic_X_col.drop(['Name','Ticket'])
house_X, house_Y = house_train_df[house_X_col].copy(), house_train_df[house_Y_col].copy()
# Identify numeric and categorical columns
numeric_cols = house_train_df[house_X_col].select_dtypes(include=np.number).columns.tolist()
categorical_cols = house_train_df[house_X_col].select_dtypes(exclude=np.number).columns.tolist()
# Impute and scale numeric columns
imputer = SimpleImputer().fit(house_train_df[numeric_cols])
house_X[numeric_cols] = imputer.transform(house_X[numeric_cols])


scaler = MinMaxScaler().fit(house_X[numeric_cols])
house_X[numeric_cols] = scaler.transform(house_X[numeric_cols])
# One-hot encode categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(house_X[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
house_X[encoded_cols] = encoder.transform(house_X[categorical_cols])
house_X = house_X[numeric_cols + encoded_cols]
house_X_Train, house_X_Test, house_Y_Train, house_Y_Test = train_test_split(house_X, house_Y, test_size = 0.25,random_state = 42)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-4-4515fd344142>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  house_X[encoded_cols] = encoder.transform(house_X[categorical_cols])
<ipython-input-4-4515fd344142>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  house_X[encoded_cols] = encoder.t

In [5]:
!pip install -U pymoo
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem, get_reference_directions, get_sampling, get_crossover, get_mutation, get_termination
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.operators.sampling.lhs import LHS
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [6]:
def test_params(**params):
    model = LGBMRegressor(random_state=42, n_jobs=-1, **params).fit(house_X_Train, house_Y_Train)
    train_rmse_score = mean_squared_error(house_Y_Train, model.predict(house_X_Train), squared=False)
    val_rmse_score = mean_squared_error(house_Y_Test ,model.predict(house_X_Test), squared=False)
    return train_rmse_score, val_rmse_score

In [7]:
# define the hyperparameter optimization problem
class HyperparameterOptimizationProblem(Problem):

    def __init__(self,level):
        # define the lower and upper bounds of the hyperparameters
        # n_estimators: number of trees in the forest (integer)
        # max_depth: maximum depth of each tree (integer)
        # max_features: maximum number of features (integer)
        # min_samples_leaf: minimum number of samples required to be at a leaf node (integer)
        xl = np.array([10, 2, .01])
        xu = np.array([600, 16, .9])
        self.level=level

        # initialize the problem with 4 variables and 2 objectives
        super().__init__(n_var = 3, n_obj = 1, xl = xl, xu = xu)

    def _evaluate(self, x, out, *args, **kwargs):
        # evaluate each solution (each row of x)
        f = np.zeros((x.shape[0], self.n_obj))
        for i in range(x.shape[0]):
          if self.level==0:
            # get the hyperparameters
            n_estimators = int(x[i, 0])*3
            max_depth = int(x[i, 1])
            #min_samples_split = int(x[i, 2])*2
            #min_samples_leaf = int(x[i, 3])
            #min_weight_fraction_leaf = int(x[i, 4])
            learning_rate = float(x[i, 2])
          else:
            # get the hyperparameters
            n_estimators = int(x[i, 0])
            max_depth = int(x[i, 1])
           # min_samples_split = int(x[i, 2])
            #min_samples_leaf = int(x[i, 3])
            #min_weight_fraction_leaf = int(x[i, 4])
            learning_rate = float(x[i, 2])


            # build and train the random forest model
            model = LGBMRegressor(n_estimators=n_estimators,
                                           max_depth=max_depth,
                                           learning_rate = learning_rate,
                                           random_state=42,
                                           n_jobs = -1)
            model.fit(house_X_Train, house_Y_Train)

            # predict on the test set
            y_pred = model.predict(house_X_Test)

            # calculate the accuracy, f1 and ROC/AUC score as the objectives
            #f[i, 0] = -accuracy_score(house_Y_Test, y_pred) # negate because we want to maximize
            #f[i, 1] = -f1_score(house_Y_Test, y_pred) # negate because we want to maximize
            # f[i, 2] = -roc_auc_score(titanic_Y_Test, y_pred) # negate because we want to maximize
            f[i, 0] = mean_squared_error(house_Y_Test, y_pred, squared = False)


        # assign the objectives to the output dictionary
        out["F"] = f

In [8]:
%%time
# create an instance of the problem
problem = HyperparameterOptimizationProblem(level=0)
problem1 = HyperparameterOptimizationProblem(level=1)

# create an instance of NSGA-III algorithm
algorithm = NSGA2(
    pop_size= 50,
    #ref_dirs=get_reference_directions("das-dennis", 3, n_partitions=12),
    # sampling=get_sampling("int_random"),
    sampling=LHS(),
    selection = RandomSelection(),
    # crossover=get_crossover("int_sbx", prob=0.9, eta=15),
    crossover = SBX(prob=1.0, prob_var=1.0),
    mutation=PolynomialMutation(prob=1.0),
    eliminate_duplicates=True)

algorithm1 = NSGA2(
    pop_size= 25,
    #ref_dirs=get_reference_directions("das-dennis", 3, n_partitions=12),
    # sampling=get_sampling("int_random"),
    sampling=LHS(),
    selection = RandomSelection(),
    # crossover=get_crossover("int_sbx", prob=0.9, eta=15),
    crossover = SBX(prob=0.8, prob_var=0.7),
    mutation=PolynomialMutation(prob=0.6),
    eliminate_duplicates=True)

# create an instance of termination criterion
# termination = get_termination("n_gen", 50)

# early stop
termination = DefaultMultiObjectiveTermination(

    n_max_gen=50
)

# perform the optimization
res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               save_history=True,
               verbose=True)
res1 = minimize(problem1,
               algorithm1,
               termination,
               seed=42,
               seed_population=res.pop,
               save_history=True,
               verbose=True)

# print the results
print(f"Best solution found: \nX = {res1.X.astype(int)} \nF = {-res1.F}") # negate F because we maximized

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [9]:
# print the results
print(f"Best solution found: \nX = {res1.X.astype(int)} \nF = {-res1.F}")

Best solution found: 
X = [547   3   0] 
F = [-26141.96472802]
